<a href="https://colab.research.google.com/github/lucaskydelima/Optimization-with-Python-Pyomo/blob/main/S1Example3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyomo
! apt-get -y -qq install glpk-utils

In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [7]:
# Defining the model
# Using set definition - S1Example1 - a revisit
model = pyo.ConcreteModel()

# Set
model.i = pyo.Set(initialize=('HIW', 'HIM'))
model.j = pyo.Set(initialize=('Comedy', 'Football'))


# Params
model.cost = pyo.Param(model.j, initialize={'Comedy': 50000, 'Football': 100000})
cost = model.cost
model.minimum = pyo.Param(model.i, initialize={'HIW': 28,  'HIM': 24})
minimum = model.minimum
model.viewers = pyo.Param(model.i, model.j, initialize={('HIW','Comedy'): 7, ('HIW','Football'): 2,
                                                        ('HIM','Comedy'): 2, ('HIM','Football'): 12})
viewers = model.viewers

# Decision Variables
model.x = pyo.Var(model.j, domain=pyo.NonNegativeReals)
x = model.x

# Objective Function
def Objective_rule(model, j):
  return sum(x[j]*cost[j] for j in model.j)
model.obj = pyo.Objective(rule=Objective_rule, sense=pyo.minimize)

def Constraint(model, i):
  return sum(viewers[i,j]*x[j] for j in model.j) >= minimum[i]
model.const = pyo.Constraint(model.i, rule=Constraint, doc = 'Viewership requirement')

# Solve
results = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

results.write()
print('\n RESULTS \n')
print('Cost of advertisement campaign =', model.obj())
print('Number of comedy ads purchased  =', x['Comedy']())
print('Number of football ads purchased =', x['Football']())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 320000.0
  Upper bound: 320000.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 5
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.018023252487182617
# ----------------------------------------------------------
#   Solution Information
# ------------------------------